In [1]:
import os
import re
import json
import pandas as pd
from src.utils import read_config
from prompts import gemini_prompts
import google.generativeai as genai
import google.ai.generativelanguage as glm

from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS, DistanceStrategy
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain_google_genai import (
    ChatGoogleGenerativeAI,
    HarmBlockThreshold,
    HarmCategory,
)

from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Building prefix dict from d:\github\project_envelope\backend\dict\dict.big.txt ...
Loading model from cache C:\Users\Sean\AppData\Local\Temp\jieba.u72d3e31b19648520dde69c2a4ddf48a5.cache
Loading model cost 1.112 seconds.
Prefix dict has been built successfully.
e:\miniconda3\envs\project_envelop\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 0609 notes
1. Add KG(knowledge graph) as part of metadata
2. Use SelfQueryRetriver to get the top_k answer
3. Override the `_get_docs_with_query` to use the `similarity_search_with_score` method(or with `similarity_search_with_score_by_vector`)

In [2]:
configs = read_config('.env/configs.json')
api_key =configs['g_key']
os.environ["GOOGLE_API_KEY"] = api_key
genai.configure(api_key=api_key)

In [3]:
model_name = "sentence-transformers/distiluse-base-multilingual-cased-v1"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

e:\miniconda3\envs\project_envelop\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# vector_db_path = "./vector_db_2/"

# # load FAISS
# vectorstore = FAISS.load_local(folder_path=vector_db_path, allow_dangerous_deserialization=True, embeddings=embeddings)

In [5]:
# chromadb
vector_db_path = "./chroma_db/"
chroma_collection_name = "correction_v1"
vectorstore = Chroma(persist_directory=vector_db_path, collection_name=chroma_collection_name, embedding_function=embeddings)

In [6]:
vectorstore._collection.count()

75

### Function for getting triple

In [7]:
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    convert_system_message_to_human=True,
    safety_settings={
        HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE,
        HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
    },
)

In [9]:
metadata_field_info = [
    AttributeInfo(
        name='source',
        description='the name of the document',
        type='string'
    ),
    AttributeInfo(
        name='target',
        description='the receiver of the document',
        type='string',
    ),
    AttributeInfo(
        name='reason',
        description='the reason of having this document',
        type='string',
    ),
    AttributeInfo(
        name='fact',
        description='the fact of the document',
        type='string',
    ),
    AttributeInfo(
        name='keywords',
        description='the keywords of the reason part of the document',
        type='string'
    ),
    AttributeInfo(
        name='relationship_between_entities',
        description='the relations between entities in the document',
        type='string'
    )
]

In [10]:
top_k = 3
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_contents="test",
    metadata_field_info=metadata_field_info,
    k=top_k,
    search_type='similarity',
    search_kwagrs={"k":top_k},
    enable_limit=True
)

# additional query
# mixed_search_string = additional_query + "與" + query

In [11]:
query = "行政院原住民"
retriever.get_relevant_documents(query)

e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


[Document(page_content='行政院客家委員會、行政院原住民族委員會及僑務委員會將客家電視、原住民電視及臺灣宏觀電視等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，爰依法提案糾正', metadata={'fact': '行政院客家委員會、行政院原住民族委員會及僑務委員會(下稱客委會、原民會及僑委會)將客家電視、原住民電視及臺灣宏觀電視(下稱客視、原視及宏觀電視)等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會(下稱公視基金會)辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局(下稱新聞局)坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，茲提出事實與理由於后一、據陳訴人指陳略以，95年無線公股處理條例於立法院三讀通過，目的在使黨、政、軍退出媒體，95年7月17日行政院召開「研商『無電視事業公股處理條例』規範原民台、客家台及宏觀頻道交由公共電視台製播2相關機關間溝通協調機制之建立問題」會議，會中決議「其製播所需經費之移撥(無政府採購法之適用問題)…」詎客委會、原民會及僑委會未依該決議辦理，仍依政府採購法，以標案形式由公視基金會辦理，實非合理又公視基金會辦理「原、客、宏觀頻道之計畫與製播」，並無獨立之經費得以辦理，而係由政府部門辦理審查、驗收，始得請領款項之情況，違反無線公股處理條例黨政軍退出媒體之精神，造成公共電視新聞自主性、專業性之箝制，詎該公股處理條例對「經費」疏漏等問題，迄未立法補正，亦造成公共電視台(下稱公視)財務經營之困境目前公視代為製播「原、客、宏觀頻道」，亦遭客委會、原民會及僑委會等延宕付款，更造成公視財務經營困難等語云云二、依據「無線電視事業公股處理條例」第14條第3項所定「政府編列預算招標採購或設置之客家電視、原住民電視、台灣宏觀電視等頻道節目之製播，應於本條例公布施行後之次年度起，『交由』公視基金會辦理」目前公視基金會係依照政府採購法之規定，定期向客委會、原民會及僑委會提出營運計畫書、工作報告、驗收作業等相關作業惟公視基金

In [12]:
from typing import Any, Dict


class CustomSelfQueryRetriever(SelfQueryRetriever):
    def _get_docs_with_query(
        self, query: str, search_kwargs: Dict[str, Any]
    ):
        """Get docs, adding score information."""
        docs, scores = zip(
            *vectorstore.similarity_search_with_score(query, **search_kwargs)
        )
        for doc, score in zip(docs, scores):
            doc.metadata["score"] = score

        return docs

In [13]:
document_content_description = 'agent'
retriever = CustomSelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)


retriever.invoke("行政院原住民")


e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


(Document(page_content='行政院客家委員會、行政院原住民族委員會及僑務委員會將客家電視、原住民電視及臺灣宏觀電視等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，爰依法提案糾正', metadata={'fact': '行政院客家委員會、行政院原住民族委員會及僑務委員會(下稱客委會、原民會及僑委會)將客家電視、原住民電視及臺灣宏觀電視(下稱客視、原視及宏觀電視)等頻道節目之製播，依政府採購法委由財團法人公共電視文化事業基金會(下稱公視基金會)辦理，雖非無據，惟造成公視基金會與客委會、原民會及僑委會針對上開頻道節目之製播各有立場、互生歧見，致紛爭不斷，詎行政院新聞局(下稱新聞局)坐視上開爭端，非但未積極研修法令，亦未尋求妥適解決辦法，核有怠失，茲提出事實與理由於后一、據陳訴人指陳略以，95年無線公股處理條例於立法院三讀通過，目的在使黨、政、軍退出媒體，95年7月17日行政院召開「研商『無電視事業公股處理條例』規範原民台、客家台及宏觀頻道交由公共電視台製播2相關機關間溝通協調機制之建立問題」會議，會中決議「其製播所需經費之移撥(無政府採購法之適用問題)…」詎客委會、原民會及僑委會未依該決議辦理，仍依政府採購法，以標案形式由公視基金會辦理，實非合理又公視基金會辦理「原、客、宏觀頻道之計畫與製播」，並無獨立之經費得以辦理，而係由政府部門辦理審查、驗收，始得請領款項之情況，違反無線公股處理條例黨政軍退出媒體之精神，造成公共電視新聞自主性、專業性之箝制，詎該公股處理條例對「經費」疏漏等問題，迄未立法補正，亦造成公共電視台(下稱公視)財務經營之困境目前公視代為製播「原、客、宏觀頻道」，亦遭客委會、原民會及僑委會等延宕付款，更造成公視財務經營困難等語云云二、依據「無線電視事業公股處理條例」第14條第3項所定「政府編列預算招標採購或設置之客家電視、原住民電視、台灣宏觀電視等頻道節目之製播，應於本條例公布施行後之次年度起，『交由』公視基金會辦理」目前公視基金會係依照政府採購法之規定，定期向客委會、原民會及僑委會提出營運計畫書、工作報告、驗收作業等相關作業惟公視基金

In [14]:
result = retriever.invoke("包含違反政府採購法")


e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [15]:
for i, page in enumerate(result):
    print(page.page_content)

臺北市立聯合醫院頻以小額採購名義逕洽特定廠商採購中藥品，未適時依政府採購法辦理採購；且部分品項採購數量超過小額採購上限10萬元用量，肇致後續發生無法核銷貨款之情事；另辦理案號10807緊急採購，違反政府採購法第105條第1項第2款規定；廠商開立之發票金額與決標單價不一致，核銷時由總務室核銷承辦人手動將ERP系統內帶入之契約單價改為發票上之金額，處理過程便宜行事；又因可歸責A生技製藥股份有限公司之事由與該公司終止契約，卻未依契約約定扣留原廠商貨款，且就機關所受損害進行求償之追討作業延宕；辦理108-109年度中藥品之採購，發生多項採購作業違反相關規定及錯誤行為態樣，內部控制制度之作業已然失靈，均核有違失，爰依法提案糾正
南投縣政府辦理101年全民運動會相關採購案，未於招標前確認採購標的性質是否不宜以最低標決標，逕採準用最有利標方式辦理，屬「最有利標作業手冊」所列最有利標錯誤行為態樣之一，核與政府採購法、政府採購法施行細則、最有利標作業手冊、政府採購決標方式認定原則等規定不符；又該府辦理「101年全民運動會典禮及各項表演相關採購」等4件勞務採購案，核有經費核定、支用與核銷均未具嚴謹且作業遲延等情事；投入總經費9千萬餘元，惟相關設施、財產或物品，於活動結束後，或囤放倉庫保管或逕行拆除，未能妥善規劃運用，促進再利用等，均有違失，爰依監察法第24條提案糾正
台灣電力股份有限公司高雄興達電廠辦理「興#3、4機煙囪彩繪及修繕工程」案，招標公告對於投標廠商資格為「特定資格」之限制，與政府採購法規定未合；且發現疑涉遭圍標等影響採購公正之違反法令行為，竟續予開標決標，未落實政府採購法相關規定；又僅依目測估計及拍照比對鋼架銹蝕情形，即簽辦並編列1億2,000萬元預算進行大規模修繕，相關評估作業顯屬草率，且有規避相關採購案須呈報上級核定之嫌；嗣執行過程因變更物料載運方式及底漆施作面積增加，辦理2次契約變更，對於監造單位工程評估之審查，未盡確實，經核均有違失，爰依法提案糾正
衛生福利部臺北醫院及基隆醫院辦理採購案涉有違失，惟未依相關法令規定懲處相關採購人員，且處分結果與違失情節顯不相當；而衛生福利部竟未加以檢討，復未能指正處分缺失，顯未善盡督導與審核職責，均有怠失另衛生福利部所屬醫療機構辦理採購案件，各作業階段均有未符政府採購法相關規定情事，影響採購公正性，且該部未落實採購稽核作業，亦未加強

In [16]:
page.metadata

{'fact': '衛生福利部（原行政院衛生署，於102年7月23日改制，下稱衛福部）所屬臺北醫院、基隆醫院、樂生療養院等3機關辦理採購案件，作業過程涉有採購缺失，案經本院調查結果，衛福部及所屬醫療機構對本案之相關處理情形，核有下列違失之處一、臺北醫院及基隆醫院辦理「全自動酵素免疫分析儀之試劑」等4件採購案涉有違失，惟該二醫院對相關採購人員未依相關法令規定懲處，且處分結果與違失情節顯不相當，殊有可議；另衛生福利部未能檢討處分結果與違失情節是否相當，亦未提出審核意見另為妥適處理，復未能指正所屬該二醫院處分缺失，顯未善盡督導與審核職責，均有怠失(一)按審計部針對衛福部暨所屬醫院、療養院於民國(下同)94至98年之採購案件，篩選其中具特定投標2廠商組合、投標或得標率偏高、決標標比偏高、競標廠商間互任董監事、變更設計次數頻繁及分批辦理公告金額以上採購等情形，計有臺北醫院、基隆醫院、樂生療養院等3機關辦理「醣化血色素分析儀試劑合作案壹式」等共30案（每一醫院各10案），總決標金額為新臺幣（下同）2億50萬餘元，發現採購過程相關缺失如下１、底價核定過程涉有疑義及投標廠商異常關聯部分調查臺北醫院辦理「全自動酵素免疫分析儀之試劑」案，核有採分項報價辦理之採購，得標廠商優先減價之27個分項報價，均與核定底價表完全相同，底價核定過程涉有疑義另基隆醫院辦理「全自動免疫分析儀試劑」、「96年度防火區劃整修工程」及「97年度院區整修工程」等3案，核有廠商投標文件內容筆跡雷同或存有異常關聯情事，機關未及時發現，深入查證，屬「政府採購法錯誤行為態樣」列載，可能有圍標之嫌或宜注意之現象等缺失２、採購作業階段合規性缺失審計部抽查該30件採購案，於招標、底價訂定、決標、驗收及結算等採購作業階段，核有「投標須知履約保證金有效期限與規定不符」、「招標公告未刊載受理疑義、異議及檢舉單位之名稱、地址、電話及傳真」、「分批辦理公告金額以上之採購」等18項缺失態樣，缺失合計共77件次(二)據審計部調查發現，臺北醫院辦理「全自動酵素免疫分析儀之試劑」採購案，於94年3月24日決標，惟得標廠商優先減價之27個分項報價，與核定底價表完全相同，底價核定及報價過程顯有異常案經法務部調查局新北市調查處偵辦，現移臺灣新北3地方法院檢察署辦理中惟本案底價係參考以往決標價格，將各分項金額予以調整，投標廠商應無法預先得知各分項底價，

In [22]:
query = '行政院原住民族委員會文化園區管理局'

In [17]:
CONCLUSION_PROMPT="""
### INSTRUCTION: 說明查詢的內容 QUERY 與 REF 中的之間的關係。若找不到合理的關係則說沒有明確的相關性，不要亂說
### QUERY: {query}
### REF: {reference}
### ASSISTANT: 
"""

In [31]:
# response_data = {}
# for i, page in enumerate(result):
#     print(page.metadata['source'])
#     # summary_prompt = PromptTemplate.from_template(gemini_prompts.SUMMARY_PROMPT)
#     # chain = summary_prompt | llm
#     # summary_data = {
#     #     "reference": page.page_content
#     # }

#     # summary_response = chain.invoke(summary_data)
    
#     # pattern = r'\*\*摘要：\*\*'
#     # if re.search(pattern, summary_response.content):
#     #     re.sub(pattern, '', summary_response.content)
        
#     suffix = "### ASSISTANT: "
#     input_variables = ["query", "reference"]

#     conclusion_prompt = PromptTemplate.from_examples(CONCLUSION_PROMPT, suffix, input_variables)
#     chain = conclusion_prompt | llm
#     conclusion_data = {
#         "query": query,
#         "reference": page.page_content
#     }
#     conclusion_response = chain.invoke(conclusion_data)

#     # response_data[i] = {
#     #     "summary": summary_response.content,
#     #     "source": page.metadata['source'],
#     #     "score": f"{page.metadata['score']:.2f}",
#     #     "target": page.metadata['target'],
#     #     # "conclusion": conclusion_response.content
#     # }
    



In [ ]:
conclusion_data

{'query': '行政院原住民',
 'reference': '臺北市立聯合醫院頻以小額採購名義逕洽特定廠商採購中藥品，未適時依政府採購法辦理採購；且部分品項採購數量超過小額採購上限10萬元用量，肇致後續發生無法核銷貨款之情事；另辦理案號10807緊急採購，違反政府採購法第105條第1項第2款規定；廠商開立之發票金額與決標單價不一致，核銷時由總務室核銷承辦人手動將ERP系統內帶入之契約單價改為發票上之金額，處理過程便宜行事；又因可歸責A生技製藥股份有限公司之事由與該公司終止契約，卻未依契約約定扣留原廠商貨款，且就機關所受損害進行求償之追討作業延宕；辦理108-109年度中藥品之採購，發生多項採購作業違反相關規定及錯誤行為態樣，內部控制制度之作業已然失靈，均核有違失，爰依法提案糾正'}

In [26]:
query = '受文對象是行政院原住民族委員會文化園區管理局'

In [29]:
class CustomSelfQueryRetriever(SelfQueryRetriever):
        def _get_docs_with_query(
            self, query: str, search_kwargs: Dict[str, Any]
        ):
            """Get docs, adding score information."""
            docs, scores = zip(
                *vectorstore.similarity_search_with_score(query, **search_kwargs)
            )
            for doc, score in zip(docs, scores):
                doc.metadata["score"] = 1-score

            return docs
        
document_content_description = 'agent'
retriever = CustomSelfQueryRetriever.from_llm(
        llm,
        vectorstore,
        document_content_description,
        metadata_field_info,
)
docs = retriever.invoke(query)
print(query)
response_data = {}
for i, page in enumerate(docs):
    print(page.metadata['source'])
    summary_prompt = PromptTemplate.from_template(gemini_prompts.SUMMARY_PROMPT)
    chain = summary_prompt | llm
    summary_data = {
        "reference": page.page_content
    }

    summary_response = chain.invoke(summary_data)
    
    pattern = r'\*\*摘要：\*\*'
    if re.search(pattern, summary_response.content):
        re.sub(pattern, '', summary_response.content)
        
        
    # conclusion_prompt = PromptTemplate.from_examples(gemini_prompts.CONCLUSION_PROMPT)
    # chain = conclusion_prompt | llm
    # conclusion_data = {
    #     "query" : query,
    #     "reference": page.page_content
    # }
    # conclusion_response = chain.invoke(conclusion_data)

    response_data[i] = {
        "summary": summary_response.content,
        "source": page.metadata['source'],
        "score": f"{page.metadata['score']:.2f}",
        "target": page.metadata['target'],
        # "conclusion": conclusion_response.content
    }

e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


受文對象是行政院原住民族委員會文化園區管理局
49990.pdf


e:\miniconda3\envs\project_envelop\lib\site-packages\langchain_google_genai\chat_models.py:344: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [30]:
response_data

{0: {'summary': '原民會文化園區管理局採購案爆發弊案，包括約僱人員包勝雄、主任秘書浦忠義、專員蔡美琴涉嫌浮報價額、核銷不實，經司法判決確定有罪。調查發現，採購與驗收過程草率。此外，採購邵族樂舞製作勞務時，竟以限制性招標方式逕洽邵族文化發展協會議價簽約，但該協會涉嫌違法轉包，而管理局未依法辦理，也有違法情事。',
  'source': '49990.pdf',
  'score': '1.03',
  'target': '行政院原住民族委員會文化園區管理局'}}